In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Use CPU for inference

In [ ]:
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed
# from datasets import load_dataset
# from tqdm import tqdm

# def generate_prompt(train_examples, test_example):
#     prompt_template_head = "Your task is to choose the correct completion for a given sentence from 4 available options.\n"
#     prompt_template_example = "Sentence: '{}', Return the most likely ending for this sentence from these 4 options: {}. Answer: {}"

#     prompt = prompt_template_head
#     for idx in range(5):
#         sample = train_examples[idx]
#         ctx = sample["ctx"]
#         endings = sample["endings"]
#         answer = endings[int(sample["label"])]

#         prompt += prompt_template_example.format(ctx, endings, answer)

#     prompt += prompt_template_example.format(
#         test_example["ctx"], test_example["endings"], ""
#     )
#     return prompt

# def opt_few_shot_pipeline():
#     set_seed(42)

#     # Load the OPT-350M model and tokenizer
#     model_name = "facebook/opt-350m"
#     tokenizer = AutoTokenizer.from_pretrained(model_name)
#     model = AutoModelForCausalLM.from_pretrained(model_name)
#     hellaswag = load_dataset("Rowan/hellaswag")
#     train_examples = hellaswag["train"]
#     test_examples = hellaswag["validation"]
# #     for key in hellaswag.keys() :
# #         print(key)

#     correct_answers = 0
#     total_examples = len(test_examples)
# #     print(test_examples[3])

#     # Use tqdm to show progress bar
#     for test_example in tqdm(test_examples):
#         prompt = generate_prompt(train_examples, test_example)
#         inputs = tokenizer(prompt, return_tensors="pt")

#         # Generate
#         generate_ids = model.generate(inputs.input_ids, max_length=800)
#         answer = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

#         # Evaluate the answer
# #         print(test_example["label"])
#         if answer == test_example["endings"][int(test_example["label"])]:
#             correct_answers += 1
# #         print(correct_answers)
#     accuracy = correct_answers / total_examples
#     print("Accuracy:", accuracy)

# if __name__ == "__main__":
#     opt_few_shot_pipeline()


## Use GPU for inference

In [1]:
!pip install transformers accelerate optimum


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.5/402.5 kB 20.8 MB/s eta 0:00:00


In [17]:
def last_occurrence_indices(sentence):
    first_index = sentence.rfind("Answer: ")
    
    if first_index != -1:
        last_index = first_index + 8
        return last_index
    else:
        return None
    
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed
from datasets import load_dataset
from tqdm import tqdm
from optimum.bettertransformer import BetterTransformer

# from optimum.bettertransformer import BetterTransformer


def generate_prompt(train_examples, test_example):
    prompt_template_head = "Your task is to choose the correct completion for a given sentence from 4 available options.\n"
    prompt_template_example = "Sentence: '{}', Return the most likely ending for this sentence from these 4 options:\n{}\n{}\n{}\n{}\nAnswer: {}"

    prompt = prompt_template_head
    for idx in range(5):
        sample = train_examples[idx]
        ctx = sample["ctx"]
        endings = sample["endings"]
        answer = endings[int(sample["label"])]

        prompt += prompt_template_example.format(ctx,endings[0],endings[1],endings[2],endings[3],answer)
        prompt +="\n"
    prompt += prompt_template_example.format(
        test_example["ctx"], test_example["endings"][0], test_example["endings"][1], test_example["endings"][2], test_example["endings"][3], ""
    )
    return prompt

def opt_few_shot_pipeline():
    set_seed(42)

    # Load the OPT-350M model and tokenizer
    model_name = "facebook/opt-350m"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16).to("cuda")
    # convert the model to BetterTransformer
    model = BetterTransformer.transform(model)

    hellaswag = load_dataset("Rowan/hellaswag")
    train_examples = hellaswag["train"]
    test_examples = hellaswag["validation"]
#     for key in hellaswag.keys() :
#         print(key)

    correct_answers = 0
    total_examples = len(test_examples)
#     print(test_examples[3])

    # Use tqdm to show progress bar
    ctr = 0
    for test_example in tqdm(test_examples):
        prompt = generate_prompt(train_examples, test_example)
        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

        # Generate
#         print("Prompt Length: ",inputs.data['input_ids'].size(dim=1),"\n")
        input_length = inputs.data['input_ids'].size(dim=1) + 30
        generate_ids = model.generate(inputs.input_ids, max_length = input_length)
        answer = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
#         print(answer)
        end_index = last_occurrence_indices(answer)
#         print("GENERATED ANSWER:",answer[end_index:])
#         print("ACTUAL ANSWER:",test_example["endings"][int(test_example["label"])])

        # Evaluate the answer
#         print(test_example["label"])
        if answer[end_index:] == test_example["endings"][int(test_example["label"])]:
            correct_answers += 1
        if ctr % 1000 == 0:
            print("Correct Answers:", correct_answers)
        ctr+=1
    accuracy = correct_answers / total_examples
    print("Accuracy:", accuracy)


if __name__ == "__main__":
    opt_few_shot_pipeline()

The BetterTransformer implementation does not support padding during training, as the fused kernels do not support attention masks. Beware that passing padded batched data during training may result in unexpected outputs. Please refer to https://huggingface.co/docs/optimum/bettertransformer/overview for more details.


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 1/10042 [00:00<1:22:15,  2.03it/s]

Correct Answers: 0


 10%|▉         | 1001/10042 [08:05<1:12:53,  2.07it/s]

...

Correct Answers: 1


100%|█████████▉| 10001/10042 [1:21:52<00:20,  2.02it/s]

Correct Answers: 1


100%|██████████| 10042/10042 [1:22:12<00:00,  2.04it/s]

Accuracy: 9.958175662218681e-05


In [11]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed
from datasets import load_dataset
from tqdm import tqdm
from optimum.bettertransformer import BetterTransformer

# from optimum.bettertransformer import BetterTransformer


def generate_prompt(train_examples, test_example):
    # prompt_template_head = "Your task is to choose the correct completion for a given sentence from 4 available options.\n"
    # prompt_template_example = "Sentence: '{}', Return the most likely ending for this sentence from these 4 options: {}. Answer: {}"
    prompt_template_head = "Your task is to choose the correct completion for a given sentence from 4 available options.\n"
    prompt_template_example = "Sentence: '{}', Return the most likely ending for this sentence from these 4 options:\n A. '{}'\n B. '{}'\n C. '{}'\n D. '{}'\n Answer: {}"

    prompt = prompt_template_head
    for idx in range(4,10):  # Adjusted to display 4 training examples
        sample = train_examples[idx]
        ctx = sample["ctx"]
        endings = sample["endings"]
        label = int(sample["label"])
        answer = chr(ord('A') + label)  # Convert numeric label to alphabetic choice
        answer+="\n"
        # Display choices as A, B, C, D
        prompt += prompt_template_example.format(ctx, endings[0], endings[1], endings[2], endings[3], answer)

    # Display the test example
    label = ""  # Test example doesn't have a label
    prompt += prompt_template_example.format(
        test_example["ctx"], test_example["endings"][0], test_example["endings"][1], test_example["endings"][2], test_example["endings"][3], label
    )
    return prompt

def opt_few_shot_pipeline():
    set_seed(42)

    # Load the OPT-350M model and tokenizer
    model_name = "facebook/opt-350m"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16).to("cuda")
    # convert the model to BetterTransformer
    model = BetterTransformer.transform(model)

    hellaswag = load_dataset("Rowan/hellaswag")
    train_examples = hellaswag["train"]
    test_examples = hellaswag["validation"]
#     for key in hellaswag.keys() :
#         print(key)

    correct_answers = 0
    total_examples = len(test_examples)
#     print(test_examples[3])

    # Use tqdm to show progress bar
    ctr = 0
    for test_example in tqdm(test_examples):
        prompt = generate_prompt(train_examples, test_example)
        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

        # Generate
        generate_ids = model.generate(inputs.input_ids, max_length=700)
        answer = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
        print(answer)
        # Evaluate the answer
#         print(test_example["label"])
        if answer[-2] == test_example["endings"][int(test_example["label"])]:
            correct_answers += 1
#         if ctr % 1000 == 0:
#             print("Correct Answers:", correct_answers)
#         print("Correct Answers:", correct_answers)
        ctr+=1
    accuracy = correct_answers / total_examples
    print("Accuracy:", accuracy)


if __name__ == "__main__":
    opt_few_shot_pipeline()


The BetterTransformer implementation does not support padding during training, as the fused kernels do not support attention masks. Beware that passing padded batched data during training may result in unexpected outputs. Please refer to https://huggingface.co/docs/optimum/bettertransformer/overview for more details.


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/10042 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1136: UserWarning: Input length of input_ids is 848, but `max_length` is set to 700. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
  0%|          | 1/10042 [00:00<19:29,  8.59it/s]/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1136: UserWarning: Input length of input_ids is 857, but `max_length` is set to 700. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
  0%|          | 2/10042 [00:00<18:33,  9.02it/s]

Your task is to choose the correct completion for a given sentence from 4 available options.
Sentence: 'The man in the center is demonstrating a hairstyle on the person wearing the blue shirt. the man in the blue shirt', Return the most likely ending for this sentence from these 4 options:
 A. 'is standing on the sponge cutting the hair of the person wearing the blue shirt.'
 B. 'is doing the hairstyle with his hand and the hairspray.'
 C. 'sits on the chair next to the sink.'
 D. 'is being shown eye to eye.'
 Answer: C
Sentence: 'The man in the blue shirt sits on the chair next to the sink. The other man begins washing his hair. he', Return the most likely ending for this sentence from these 4 options:
 A. 'drops the water faucet in the sink.'
 B. 'then walks over to the sink and smiles while shaking his wet hair.'
 C. 'turns on the water and gets out of the chair.'
 D. 'scrubs in the shampoo and then washes it off.'
 Answer: D
Sentence: 'The man in the blue shirt sits on the chair ne

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1136: UserWarning: Input length of input_ids is 903, but `max_length` is set to 700. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
  0%|          | 3/10042 [00:00<19:21,  8.65it/s]/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1136: UserWarning: Input length of input_ids is 843, but `max_length` is set to 700. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
  0%|          | 4/10042 [00:00<18:35,  9.00it/s]

...

Your task is to choose the correct completion for a given sentence from 4 available options.
Sentence: 'The man in the center is demonstrating a hairstyle on the person wearing the blue shirt. the man in the blue shirt', Return the most likely ending for this sentence from these 4 options:
 A. 'is standing on the sponge cutting the hair of the person wearing the blue shirt.'
 B. 'is doing the hairstyle with his hand and the hairspray.'
 C. 'sits on the chair next to the sink.'
 D. 'is being shown eye to eye.'
 Answer: C
Sentence: 'The man in the blue shirt sits on the chair next to the sink. The other man begins washing his hair. he', Return the most likely ending for this sentence from these 4 options:
 A. 'drops the water faucet in the sink.'
 B. 'then walks over to the sink and smiles while shaking his wet hair.'
 C. 'turns on the water and gets out of the chair.'
 D. 'scrubs in the shampoo and then washes it off.'
 Answer: D
Sentence: 'The man in the blue shirt sits on the chair ne

  3%|▎         | 272/10042 [00:30<17:16,  9.43it/s]

Your task is to choose the correct completion for a given sentence from 4 available options.
Sentence: 'The man in the center is demonstrating a hairstyle on the person wearing the blue shirt. the man in the blue shirt', Return the most likely ending for this sentence from these 4 options:
 A. 'is standing on the sponge cutting the hair of the person wearing the blue shirt.'
 B. 'is doing the hairstyle with his hand and the hairspray.'
 C. 'sits on the chair next to the sink.'
 D. 'is being shown eye to eye.'
 Answer: C
Sentence: 'The man in the blue shirt sits on the chair next to the sink. The other man begins washing his hair. he', Return the most likely ending for this sentence from these 4 options:
 A. 'drops the water faucet in the sink.'
 B. 'then walks over to the sink and smiles while shaking his wet hair.'
 C. 'turns on the water and gets out of the chair.'
 D. 'scrubs in the shampoo and then washes it off.'
 Answer: D
Sentence: 'The man in the blue shirt sits on the chair ne

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1136: UserWarning: Input length of input_ids is 900, but `max_length` is set to 700. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
  3%|▎         | 272/10042 [00:30<18:06,  8.99it/s]


KeyboardInterrupt: 